## Storing data in a networked Postgres database

To avoid the "pg_config executable not found" error when installing the psycopg2 package, ensure that the necessary PostgreSQL libraries and headers are installed on your system. For Debian/Ubuntu systems, execute sudo apt-get install libpq-dev python3-dev. For Red Hat/CentOS/Fedora, use sudo yum install postgresql-devel python3-devel. On macOS, install PostgreSQL with Homebrew using brew install postgresql. Alternatively, you can install psycopg2-binary with pip install psycopg2-binary, which includes the required dependencies without needing compilation. However, psycopg2-binary is more suitable for development and limited testing rather than production environments. If issues persist after these steps, try reinstalling SQLAlchemy with pip install --force-reinstall sqlalchemy. Following these steps will enable smooth installation and operation of psycopg2 and SQLAlchemy for connecting to a PostgreSQL database.

In [3]:
!pip install sqlalchemy psycopg2

  Using cached SQLAlchemy-2.0.37-cp310-cp310-macosx_11_0_arm64.whl.metadata (9.6 kB)
  Using cached psycopg2-2.9.10.tar.gz (385 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 3.2 MB/s eta 0:00:00a 0:00:01
  Created wheel for psycopg2: filename=psycopg2-2.9.10-cp310-cp310-macosx_15_0_arm64.whl size=144715 sha256=77d0952fa914e3ab4b20ce97329d004c961556c496bd4fa7b67723d934862baf
  Stored in directory: /Users/kirilllisenkov/Library/Caches/pip/wheels/51/41/e0/2912ad51b01f454d26dfb26e5cc5923874656749b9e83943a8
Successfully built psycopg2


In [4]:
import exchange_calendars as xcals
import pandas as pd
from IPython.display import Markdown, display
from openbb import obb
from sqlalchemy import create_engine, text
from sqlalchemy.exc import ProgrammingError

In [5]:
obb.user.preferences.output_type = "dataframe"

Database connection parameters

In [6]:
username = ""
password = ""
host = "127.0.0.1"
port = "5432"
database = "market_data"

In [7]:
DATABASE_URL = f"postgresql://{username}:{password}@{host}:{port}/postgres"
base_engine = create_engine(DATABASE_URL)

Function to create a new database and return an engine for that database

In [8]:
def create_database_and_get_engine(db_name, base_engine):
    conn = base_engine.connect()
    conn = conn.execution_options(isolation_level="AUTOCOMMIT")

    try:
        conn.execute(text(f"CREATE DATABASE {db_name};"))
    except ProgrammingError:
        pass
    finally:
        conn.close()

    conn_str = base_engine.url.set(database=db_name)

    return create_engine(conn_str)

Create the database and get an engine for it

In [11]:
engine = create_database_and_get_engine("stock_data", base_engine)

Function to fetch historical stock data for a given symbol and date range, and add a 'symbol' column

In [12]:
def get_stock_data(symbol, start_date=None, end_date=None):
    data = obb.equity.price.historical(
        symbol,
        start_date=start_date,
        end_date=end_date,
        provider="yfinance",
    )
    data.reset_index(inplace=True)
    data["symbol"] = symbol
    return data

Function to save the fetched stock data to a PostgreSQL database

In [13]:
def save_data_range(symbol, engine, start_date=None, end_date=None):
    data = get_stock_data(symbol, start_date, end_date)
    data.to_sql("stock_data", engine, if_exists="append", index=False)

Function to save the stock data for the last trading session to a PostgreSQL database

In [14]:
def save_last_trading_session(symbol, engine):
    today = pd.Timestamp.today()
    data = get_stock_data(symbol, today, today)
    data.to_sql("stock_data", engine, if_exists="append", index=False)

Save data for multiple stock symbols in the specified date range

In [15]:
for symbol in ["SPY", "QQQ", "DIA"]:
    save_data_range(
        symbol, engine=engine, start_date="2020-06-01", end_date="2023-01-01"
    )

Read and display data for the stock symbol "SPY" from the database

In [16]:
df_1 = pd.read_sql_query("SELECT * from stock_data where symbol='SPY'", engine)
display(df_1)

,date,open,high,low,close,volume,dividend,symbol
0,2020-06-01,303.619995,306.209991,303.059998,305.549988,55758300,0.0,SPY
1,2020-06-02,306.549988,308.130005,305.100006,308.079987,74267200,0.0,SPY
2,2020-06-03,310.239990,313.220001,309.940002,312.179993,92567600,0.0,SPY
3,2020-06-04,311.109985,313.000000,309.079987,311.359985,75794400,0.0,SPY
4,2020-06-05,317.230011,321.269989,317.160004,319.339996,150524700,0.0,SPY
...,...,...,...,...,...,...,...,...
648,2022-12-23,379.649994,383.059998,378.029999,382.910004,59857300,0.0,SPY
649,2022-12-27,382.790009,383.149994,379.649994,381.399994,51638200,0.0,SPY
650,2022-12-28,381.329987,383.390015,376.420013,376.660004,70911500,0.0,SPY
651,2022-12-29,379.630005,384.350006,379.079987,383.440002,66970900,0.0,SPY


Read and display data for "SPY" where the volume is greater than 100,000,000

In [17]:
df_2 = pd.read_sql_query(
    "SELECT * from stock_data where symbol='SPY' and volume > 100000000", engine
)
display(df_2)

,date,open,high,low,close,volume,dividend,symbol
0,2020-06-05,317.230011,321.269989,317.160004,319.339996,150524700,0.000,SPY
1,2020-06-11,311.459991,312.149994,300.010010,300.609985,209243600,0.000,SPY
2,2020-06-12,308.239990,309.079987,298.600006,304.209991,194678900,0.000,SPY
3,2020-06-15,298.019989,308.279999,296.739990,307.049988,135782700,0.000,SPY
4,2020-06-16,315.480011,315.640015,307.670013,312.959991,137627500,0.000,SPY
...,...,...,...,...,...,...,...,...
149,2022-12-13,410.220001,410.489990,399.070007,401.970001,123782500,0.000,SPY
150,2022-12-14,401.609985,405.500000,396.309998,399.399994,108111300,0.000,SPY
151,2022-12-15,394.299988,395.250000,387.890015,389.630005,117705900,0.000,SPY
152,2022-12-16,385.179993,386.579987,381.040009,383.269989,119858000,1.781,SPY


The best way to build your database is to first download history. You do this once to “backfill” the historic data. Then you schedule the script to run every trading day after the market close to capture that day’s data going forward.

There are a lot of references to schedule a Python job on Mac and Windows so I won’t cover it in detail here.

Here are two that will work:

​Schedule on Mac/Linux​ : https://theautomatic.net/2020/11/18/how-to-schedule-a-python-script-on-a-mac/
​Schedule on Windows​ : https://www.jcchouinard.com/python-automation-using-task-scheduler/

**Jason Strimpel** is the founder of <a href='https://pyquantnews.com/'>PyQuant News</a> and co-founder of <a href='https://www.tradeblotter.io/'>Trade Blotter</a>. His career in algorithmic trading spans 20+ years. He previously traded for a Chicago-based hedge fund, was a risk manager at JPMorgan, and managed production risk technology for an energy derivatives trading firm in London. In Singapore, he served as APAC CIO for an agricultural trading firm and built the data science team for a global metals trading firm. Jason holds degrees in Finance and Economics and a Master's in Quantitative Finance from the Illinois Institute of Technology. His career spans America, Europe, and Asia. He shares his expertise through the <a href='https://pyquantnews.com/subscribe-to-the-pyquant-newsletter/'>PyQuant Newsletter</a>, social media, and has taught over 1,000+ algorithmic trading with Python in his popular course **<a href='https://gettingstartedwithpythonforquantfinance.com/'>Getting Started With Python for Quant Finance</a>**. All code is for educational purposes only. Nothing provided here is financial advise. Use at your own risk.